In [1]:
#Mcallen
#San Antonio
#Edinburg
#Austin
#Dallas
#San Marcos
#Corpus Christi
#Weslaco
#Boerne
#Houston

### Imports

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import urllib.parse

### HTTP Request

In [2]:
#store website in a variable
website = 'https://www.trulia.com/TX/Boerne/'

In [3]:
#set up selenium webdriver
driver = webdriver.Chrome()

#get the webpage using selenium

try:
    # Get the webpage using Selenium
    driver.get(website)
    time.sleep(5)  # Give the page some time to load

    # Scroll down to load more content
    for _ in range(20):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Adjust the delay based on the page loading speed

    # Get the updated page source after scrolling
    updated_html = driver.page_source

    # Create a BeautifulSoup object using the updated HTML
    soup = BeautifulSoup(updated_html, 'html.parser')

    # Extract the desired results with data-testid
    result_container = soup.find_all(attrs={'data-testid': True})


    # Retrieve anchor tags into a list and establish base URL
    url_part_1 = 'https://www.trulia.com'
    url_part_2 = []

    for item in result_container:
        for link in item.find_all('div', {'data-testid': 'property-card-details'}):
            url_part_2.append(link.find('a').get('href'))

finally:
    driver.quit()  # Close the browser window when done

In [4]:
# Remove spaces and duplicates from list
url_only = [x.strip(' ') for x in url_part_2]
url_only = list(dict.fromkeys(url_only))
len(url_only)

40

In [5]:
# Concatenate URL
url_joined = []
for link_2 in url_only:
    url_joined.append(urllib.parse.urljoin(url_part_1,link_2))

In [6]:
url_joined

['https://www.trulia.com/home/448-waring-welfare-rd-boerne-tx-78006-122955511',
 'https://www.trulia.com/home/106-bravo-boerne-tx-78006-224103807',
 'https://www.trulia.com/builder-community-plan/Ranches-At-Creekside-Durango-2056831339',
 'https://www.trulia.com/builder-community-plan/The-Ranches-At-Creekside-Plan-Cambridge-2052930759',
 'https://www.trulia.com/home/2130-paniolo-dr-boerne-tx-78006-2061987595',
 'https://www.trulia.com/home/1137-paniolo-dr-boerne-tx-78006-2066977747',
 'https://www.trulia.com/home/2136-paniolo-dr-boerne-tx-78006-2061987539',
 'https://www.trulia.com/home/121-Cave-Cir-409-Boerne-TX-78006-345136827',
 'https://www.trulia.com/home/8531-nichols-rim-boerne-tx-78015-89609556',
 'https://www.trulia.com/builder-community-plan/Esperanza-Englewood-2056027816',
 'https://www.trulia.com/builder-community-plan/Corley-Farms-Kisko-2053177573',
 'https://www.trulia.com/home/47-muirfield-boerne-tx-78006-224100773',
 'https://www.trulia.com/home/103-Emerald-Hl-65-Boerne-

### Get data from links

In [7]:
# address
# bedrooms
# bathrooms
# sqft
# year built
# parking
# price

In [8]:
first_link = url_joined[0]
print(first_link)

https://www.trulia.com/home/448-waring-welfare-rd-boerne-tx-78006-122955511


### Get request and Soup Object

In [9]:
response = requests.get(first_link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})

In [10]:
soup = BeautifulSoup(response.content, 'html.parser')

In [1]:
# Commenting this step out to avoid copyright
#soup

#### Put all of the attributes together and loop through results from page 1


In [12]:
address = []
bedrooms = []
bathrooms = []
area = []
year_built = []
parking = []
price = []

# Loop through all joined links
for link in url_joined:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})
    
    # Create soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Address
    try:
        address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
    except:
        address.append('')
    # Bedrooms  
    try:
        bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
    except:
        bedrooms.append('')
        
    # Bathrooms
    try:
        bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
    except:
        bathrooms.append('')
        
    # Square Feet
    try:
        area.append(soup.find('li', {'data-testid':'floor'}).get_text())
    except:
        area.append('')
    
    # Year Built
    try:
        year_built.append(soup.find('div',string='Year Built').findNext('span').get_text())
        year_built = [x.strip('Year Built: ') for x in year_built] 
    except:
        year_built.append('')
    
    # Parking
    try:
        parking.append(soup.find(string='Parking').findNext('div').get_text())
    except:
        parking.append('')
    
    # Price
    try:
        price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
    except:
        price.append('') 
    
    # Create a dictionary with results
    output = {'Address': address, 'Bathrooms': bathrooms, 'Area':area, 'Year Built':year_built, 
              'Parking':parking, 'Price':price}
    
output

{'Address': ['448 waring welfare',
  '106 BRAVO',
  'Durango Plan in Ranches at Creekside by Sitterle Homes',
  'Plan Cambridge in The Ranches at Creekside by Highland Homes',
  '2130 PANIOLO DR',
  '1137 PANIOLO DR',
  '2136 PANIOLO DR',
  '121 CAVE CIR LOT 409',
  '8531 Nichols Rim',
  'Englewood Plan in Esperanza by Scott Felder Homes',
  'Kisko Plan in Corley Farms by Centex Homes',
  '47 MUIRFIELD',
  '103 EMERALD HL LOT 65',
  'Crestview Plan in Esperanza by Scott Felder Homes',
  '518 CORDILLERA TRCE LOT 5',
  '7615 NOLAN CRK',
  "2504S Plan in Esperanza 50' by PERRY HOMES",
  '204 Woods of Boerne',
  "1984S Plan in Esperanza 50' by PERRY HOMES",
  '150 Besito',
  '120 Rocky Path',
  '162 Besito',
  '101 Brookside',
  'Lantana Plan in Esperanza by Scott Felder Homes',
  "2737W Plan in The Ranches at Creekside 55' by PERRY HOMES",
  'Plan 1792 in Shoreline Park - Classic Collection by KB Home',
  'Plan 3420 in Shoreline Park - Classic Collection by KB Home',
  'Burnet Plan in Esp

In [13]:
# Create dataframe
df = pd.DataFrame(output)
df

,Address,Bathrooms,Area,Year Built,Parking,Price
0,448 waring welfare,4 Baths,"2,642 sqft (on 6 acres)",2024,Garage,"$829,999"
1,106 BRAVO,5 Baths,"4,303 sqft (on 0.65 acres)",2017,3 Car Garage,"$889,000"
2,Durango Plan in Ranches at Creekside by Sitter...,3 Baths,"2,310 sqft",2024,Garage,"$561,000+"
3,Plan Cambridge in The Ranches at Creekside by ...,4 Baths,"2,982 sqft",2024,2 Car Garage,"$624,990+"
4,2130 PANIOLO DR,3 Baths,"1,550 sqft",2023,2 Carport Spaces,"$364,900"
5,1137 PANIOLO DR,3 Baths,"1,550 sqft",2022,2 Carport Spaces,"$364,900"
6,2136 PANIOLO DR,3 Baths,,2023,2 Carport Spaces,"$364,900"
7,121 CAVE CIR LOT 409,,,,No Info,"$415,000"
8,8531 Nichols Rim,5 Baths,"3,701 sqft (on 0.26 acres)",2011,Garage,"$650,000"
9,Englewood Plan in Esperanza by Scott Felder Homes,3 Baths,"2,508 sqft",2024,Garage,"$584,990+"


In [14]:
# Add location to dataframe
df['Location'] = 'Boerne'
df

,Address,Bathrooms,Area,Year Built,Parking,Price,Location
0,448 waring welfare,4 Baths,"2,642 sqft (on 6 acres)",2024,Garage,"$829,999",Boerne
1,106 BRAVO,5 Baths,"4,303 sqft (on 0.65 acres)",2017,3 Car Garage,"$889,000",Boerne
2,Durango Plan in Ranches at Creekside by Sitter...,3 Baths,"2,310 sqft",2024,Garage,"$561,000+",Boerne
3,Plan Cambridge in The Ranches at Creekside by ...,4 Baths,"2,982 sqft",2024,2 Car Garage,"$624,990+",Boerne
4,2130 PANIOLO DR,3 Baths,"1,550 sqft",2023,2 Carport Spaces,"$364,900",Boerne
5,1137 PANIOLO DR,3 Baths,"1,550 sqft",2022,2 Carport Spaces,"$364,900",Boerne
6,2136 PANIOLO DR,3 Baths,,2023,2 Carport Spaces,"$364,900",Boerne
7,121 CAVE CIR LOT 409,,,,No Info,"$415,000",Boerne
8,8531 Nichols Rim,5 Baths,"3,701 sqft (on 0.26 acres)",2011,Garage,"$650,000",Boerne
9,Englewood Plan in Esperanza by Scott Felder Homes,3 Baths,"2,508 sqft",2024,Garage,"$584,990+",Boerne


In [15]:
df.to_excel('dataframe_boerne.xlsx', index=False)